# TOPOPLOT

Script para el cálculo y visualización del topoplot de un paciente a partir de un archivo .edf

## Resumen

En este script se realizan los cálculos de la potencia de la señal, la ubicación de los electrodos de la medición según el modelo New York, para finalmente realizar el topoplot por medio de la biblioteca "MNE" [1]

## Entradas

Por medio de argparse se definen las siguientes entradas.

1) -i o --archivo: nombre del archivo .edf que desea utilizar, es de tipo str, ejemplo: -i sujeto_base.edf

2) -o o --figure: nombre y dirección para guardar el topoplot, ejemplo -o pictures/figure

3) -c o --config: nombre del archivo de configuración, éste debe incluir las siguientes dos variables: 
3.1) labels: de tipo lista, incluye el nombre de los electrodos según el modelo que se desea utilizar, por default se utiliza el modelo New York.
3.2) elec: array con las coordenadas de los electrodos según el modelo que se desea utilizar, por default se utiliza el modelo New York. sus dimensiones son el número de electrodos x 2. Debe ser de la misma longitud de la cantidad de elementos de "labels"

Ejemplo: -c config


## Salidas

1) fig: imagen con el topoplot para el archivo .edf

## Código

Como una breve descripción del código se puede decir que se comienza con la importación de todas las biliotecas necesarias, además de la definición de la clase general que incluye métodos comentados al principio de cada uno para cada sección del código. Finalmente se define el objeto y se realiza el llamado de los métodos.

In [ ]:
import numpy as np
import argparse
import pyedflib
import scipy.io
from scipy.spatial import Delaunay
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mne.viz import plot_topomap

class edf_topoplot(object):

	def __init__(self,):
		super(edf_topoplot, self).__init__()

	def argparse(self):
		parser = argparse.ArgumentParser()
		parser.add_argument('-i','--archivo', 
			help='nombre del archivo .edf que desea utilizar',type = str)
		parser.add_argument('-o','--figure', 
			help='ingresa la direccion para guardar el topoplot')
		parser.add_argument('-c','--config',
		 	help='nombre del archivo de configuracion. NOTA: el nombre de la variable debe ser elec')
		parsedargs = parser.parse_args()
		arc = parsedargs.archivo
		output = parsedargs.figure
		config  = parsedargs.config
		return arc, output, config

	#lee el archivo .edf y extrae la informacion principal
	def read_edf(self,arc):

		edf = pyedflib.EdfReader(arc)#archivo edf
		channels_labels = edf.getSignalLabels() #etiquetas de los canales	
		nsig = edf.getNSamples()[0] #longitud de la toma		
		nch  = edf.signals_in_file #numero de canales		
		signal = np.zeros((nch,nsig))
		for x in range(nch):
			signal[x,:] = edf.readSignal(x)	
		return signal,nch,channels_labels

	#calcula la potencia y las posiciones de los electrodos
	def calc_power(self,signal):
		magnitud = signal**2	#magnitud
		pot_signal = np.mean(magnitud,axis=1) #potencia de la senal
		# print('vector de potencias')
		# print(pot_signal)
		return pot_signal

	def calc_positions(self,config,channels_labels,nch,pot_signal):
		config_file = __import__(config)
		elec = config_file.elec 		#coordenadas de los electrodos de referencia
		lab_ref_list = config_file.labels	#lista etiquetas de referencia
		label_ref_list = []

		#organiza la lista de etiquetas de referencia para ser comparada
		for lab_ref in lab_ref_list:
			label_ref = lab_ref.upper()
			label_ref_list.append(label_ref)
		count = 0
		counter = -1
		pos = np.zeros(shape=(nch,2))

		#organiza la lista de etiquetas de la medicion para ser comparada
		for lab in channels_labels:
			labe = lab.upper()
			label1 = labe.find('-')
			label = labe[0:label1]
			exist = label in label_ref_list

			#crea un np.ndarray con las coordenadas de los electrodos existentes en la medida
			if exist is True:
				point = label_ref_list.index(label)
				coord = elec[point]
				pos[count,:] = coord
				# coord = np.asarray(coord)
			count += 1
		
		#elimina los electrodos que no fueron encontrados en la lista de referencia	
		for coord in pos:
			counter += 1
			check = np.array_equal(coord,[0.,0.])
			if check is True:
				pos = np.delete(pos,counter,0)
				pot_signal = np.delete(pot_signal,counter,0)
				counter += -1
		# print('posicion de los nodos')
		# print(pos)
		return pos,pot_signal,elec

	#elabora el topoplot
	def topoplot(self,pot_signal,pos):

		plt.title('topomap')
		fig = plot_topomap(pot_signal, pos, cmap='jet', sensors='k.', names=channels_labels, show_names=False,
				contours=0, image_interp='spline36' ,show = False)
		# plt.savefig(output)
		plt.show(fig)
		return fig

	def mesh(self,pos,elec):
		# malla de los electrodos de la muestra
		tri = Delaunay(pos)
		plt.triplot(pos[:,0], pos[:,1], tri.simplices)
		plt.plot(pos[:,0], pos[:,1], 'o')
		plt.show()

		# malla de los electrodos de New York Head Model
		tri = Delaunay(elec)
		plt.triplot(elec[:,0], elec[:,1], tri.simplices)
		plt.plot(elec[:,0], elec[:,1], 'o')
		plt.show()

if __name__ == '__main__':
	ob = edf_topoplot()
	arc,output,config = ob.argparse()
	signal,nch,channels_labels = ob.read_edf(arc)
	pot_signal = ob.calc_power(signal)
	pos, pot_signal, elec= ob.calc_positions(config,channels_labels,nch,pot_signal)
	grid, pos = ob.interpolation(pot_signal,pos,elec)
	fig = ob.topoplot(pot_signal,elec)
	_ = ob.mesh(pos,elec)



	#python topoplot.py -i sujeto_base.edf -o fig3 -c config

## Bibliotecas utilizadas

[1] MNE        http://www.martinos.org/mne/

[2] pyedflib   http://pyedflib.readthedocs.io

[3] Numpy      http://www.numpy.org/

[4] Argparse   https://docs.python.org/3/library/argparse.html

[5] Scipy      http://www.scipy.org

[6] Matplotlib http://www.matplotlib.org